In [1]:

# Import Dependencies

import numpy as np
import math

import matplotlib.pyplot as plt
# import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm

#from sklearn.neighbors import KernelDensity
# from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
# from bokeh.palettes import Dark2_5 as colors
# from bokeh.plotting import figure, show
# from bokeh.sampledata.cows import data as df
# from bokeh.io import output_notebook


In [2]:
import pandas as pd


df_course = pd.read_csv('assets/student.course.csv')
df_record = pd.read_csv('assets/student.record.csv')
df_term = pd.read_table('assets/term.table.txt', delimiter="\t").fillna('Unknown')



df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: x[-4:] if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
for term in terms_student:
    merge_record_df = pd.merge(
                        df_record,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')    
    
    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    
# df_term
# df_course
# df_record
# merge_course_df
merge_record_df

    
course_columns = ['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
                  'ANON_INSTR_ID', 'TERM', 'TERM_ID', 'TERM_DESCRIPTION', 'TERM_NAME', 'TERM_YEAR']

record_columns = ['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE',
                  'LAST_ACT_READ_SCORE', 'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE', 
                  'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1', 'STDNT_GROUP2', 'MAJOR1_DEPT', 
                  'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID', 'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM', 
                  'MAJOR3_TERM_TERM_ID', 'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME', 'MAJOR3_TERM_TERM_YEAR']   

# merge_course_df.describe()
# merge_record_df.describe()


# https://www.bls.gov/oes/tables.htm
# https://data.census.gov/table/ACSST1Y2015.S0101
# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06
merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()
# 1992 - 2015


# Occupational data by gender
# https://www.bls.gov/cps/cps_aa2015.htm

# https://www.axios.com/2020/06/30/half-us-population-unemployed
# the labor market is only at 50%?
merge_record_df.shape

(138888, 27)

# Create Student Heat Map Dataframe Data

In [3]:
df_majors_occups_map= pd.read_csv('assets/majors_occupations_map.csv')

student_data_df = df_record[['MAJOR1_DESCR','SEX','ADMIT_TERM']]
student_data_df = pd.merge(student_data_df, df_majors_occups_map, how='left', left_on='MAJOR1_DESCR',right_on='MAJOR')
student_data_df = pd.merge(student_data_df, df_term, how='left', left_on='ADMIT_TERM',right_on='TERM_ID')
student_data_df.drop(['TERM_ID','MAJOR1_DESCR'], axis=1, inplace=True)
student_data_df = student_data_df.dropna()

student_data_df = student_data_df.groupby(['TERM_YEAR','OCCUPATION','SEX'])['ADMIT_TERM'].count().reset_index()
student_data_df['TERM_YEAR'] = student_data_df['TERM_YEAR'].astype(int)
student_data_df['ESTIMATED_COMPLETION_YEAR'] = student_data_df['TERM_YEAR'] + 5
student_data_df.rename(columns={'ADMIT_TERM': 'STUDENTS_IN_RELATED_MAJOR'}, inplace=True)

# Generate total number of students for each calendar year
year_totals = student_data_df.groupby(['TERM_YEAR'])['STUDENTS_IN_RELATED_MAJOR'].sum().reset_index()
year_totals.rename(columns={'STUDENTS_IN_RELATED_MAJOR': 'TOTAL_STUDENTS_TERM_YEAR'}, inplace=True)

# Add percentage of total values for each occupation by year
student_data_df = pd.merge(student_data_df, year_totals, how='left', left_on='TERM_YEAR',right_on='TERM_YEAR')
student_data_df['PERCENTAGE']= student_data_df['STUDENTS_IN_RELATED_MAJOR']/student_data_df['TOTAL_STUDENTS_TERM_YEAR']
student_data_df.sort_values(by=['TERM_YEAR','OCCUPATION'],ascending=[False,True])
student_data_df


,TERM_YEAR,OCCUPATION,SEX,STUDENTS_IN_RELATED_MAJOR,ESTIMATED_COMPLETION_YEAR,TOTAL_STUDENTS_TERM_YEAR,PERCENTAGE
0,1996,Architecture and engineering occupations,F,289,2001,5153,0.056084
1,1996,Architecture and engineering occupations,M,640,2001,5153,0.124199
2,1996,"Arts, design, entertainment, sports, and media...",F,205,2001,5153,0.039783
3,1996,"Arts, design, entertainment, sports, and media...",M,180,2001,5153,0.034931
4,1996,Business and financial operations occupations,M,1,2001,5153,0.000194
...,...,...,...,...,...,...,...
342,2015,"Education, training, and library occupations",M,5,2020,40,0.125000
343,2015,Healthcare practitioners and technical occupat...,F,1,2020,40,0.025000
344,2015,Healthcare support occupations,F,2,2020,40,0.050000
345,2015,"Life, physical, and social science occupations",F,12,2020,40,0.300000


In [4]:
df_majors_occups_map

,MAJOR,OCCUPATION
0,Aerospace Engineering BSE,Architecture and engineering occupations
1,AfroAmerican & African St BA,"Education, training, and library occupations"
2,AfroAmerican & African St BS,"Education, training, and library occupations"
3,American Culture BA,"Education, training, and library occupations"
4,American Culture BS,"Education, training, and library occupations"
...,...,...
343,Theatre Performance BFA,"Arts, design, entertainment, sports, and media..."
344,Unclassified,Unclassified
345,Women's Studies BA,"Education, training, and library occupations"
346,Women's Studies BS,"Education, training, and library occupations"


In [5]:
import pandas as pd

# More tables here
# https://www.bls.gov/cps/tables.htm
# BLS MEDAIN WEEKLY EARNINGS https://www.bls.gov/cps/cpsaat39.htm

# https://data.bls.gov/apps/demographic-comparison-matrix/home.htm

# treemaps https://docs.bokeh.org/en/latest/docs/examples/topics/hierarchical/treemap.html

# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06

df_bls_2015_headers_1 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=0).fillna('Unknown').iloc[0]
df_bls_2015_headers_2 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=1).fillna('Unknown').iloc[0]
df_bls_2015_headers_3 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=2).fillna('Unknown').iloc[0]
df_bls_2015_headers_4 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown').iloc[0]

df_bls_2015 = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown')

df_bls_2015

tabs = ['2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003']

columns = ['level', 
           'l3','l2','l1','l0',
           'occupation', 
           'total_16_years_and_over_py', 
           'total_16_years_and_over',
           'male_16_years_and_over_py', 
           'male_16_years_and_over',
           'male_20_years_and_over_py', 
           'male_20_years_and_over',
           'female_16_years_and_over_py', 
           'female_16_years_and_over',
           'female_20_years_and_over_py', 
           'female_20_years_and_over']

# combined = list(zip(df_bls_2015_headers_1, df_bls_2015_headers_2, df_bls_2015_headers_3, df_bls_2015_headers_4))
# for row in combined:
#     print(row)


df_bls_all = pd.DataFrame()
for tab in tabs:
    df_bls_next = pd.read_excel('assets/bls_cpsaat09_2002_to_2015.xlsx', 
                                sheet_name=str(tab), 
                                header=3, 
                                ).fillna('Unknown')
    
    df_bls_next.columns = columns
    df_bls_next['year'] = str(tab)
    # df_bls_all.columns = columns.append('year')
    df_bls_all = pd.concat([df_bls_all, df_bls_next])

print(len(df_bls_all))
df_bls_all_filtered = df_bls_all[df_bls_all['occupation'] != 'Unknown']
df_bls_all_filtered_Totals = df_bls_all[(df_bls_all['level'] == 'L3') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L2 = df_bls_all[(df_bls_all['level'] == 'L2') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L0 = df_bls_all[(df_bls_all['level'] == 'L0') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]


# Check L0 Sums vs Totals
# for tab in tabs:
#     print(tab, df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == tab]['total_16_years_and_over_py'].sum(axis=0) - df_bls_all_filtered_Totals[df_bls_all_filtered_Totals['year'] == tab]['total_16_years_and_over_py'].sum(axis=0))


# print(len(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]))
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')].columns)
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]['occupation'].unique())

# Sanity Check trend
for tab in tabs:
    print(tab, df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == tab) & (df_bls_all_filtered_L0['occupation'] == 'Computer and mathematical occupations')]['total_16_years_and_over_py'].sum(axis=0))


df_bls_all_filtered



533
2015 4303.0
2014 3980.0
2013 3816.0
2012 3608.0
2011 3531.0
2010 3481.0
2009 3676.0
2008 3441.0
2007 3209.0
2006 3246.0
2005 3140.0
2004 3122.0
2003 3117.0


,level,l3,l2,l1,l0,occupation,total_16_years_and_over_py,total_16_years_and_over,male_16_years_and_over_py,male_16_years_and_over,male_20_years_and_over_py,male_20_years_and_over,female_16_years_and_over_py,female_16_years_and_over,female_20_years_and_over_py,female_20_years_and_over,year
0,L3,Unknown,Unknown,Unknown,Unknown,Total,146305.0,148834.0,77692.0,79131.0,75471.0,76776.0,68613.0,69703.0,66287.0,67323.0,2015
2,L2,Unknown,Unknown,Unknown,Unknown,"Management, professional, and related occupations",56050.0,57960.0,27119.0,28090.0,26976.0,27918.0,28931.0,29871.0,28751.0,29658.0,2015
3,L1,Unknown,Unknown,Unknown,Unknown,"Management, business, and financial operations...",23171.0,24108.0,13041.0,13589.0,13008.0,13548.0,10129.0,10519.0,10098.0,10476.0,2015
4,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Management occupations,Management occupations,16199.0,16994.0,9953.0,10340.0,9931.0,10309.0,6246.0,6654.0,6223.0,6624.0,2015
5,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Business and financial operations occupations,Business and financial operations occupations,6972.0,7114.0,3088.0,3249.0,3077.0,3238.0,3884.0,3866.0,3875.0,3851.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Construction and extraction occupations,7898.0,8114.0,7674.0,7891.0,7431.0,7636.0,224.0,223.0,215.0,214.0,2003
41,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,"Installation, maintenance, and repair occupations",4623.0,5041.0,4412.0,4830.0,4312.0,4730.0,212.0,211.0,206.0,204.0,2003
44,L2,Unknown,Unknown,Unknown,Unknown,"Production, transportation, and material movin...",18569.0,18020.0,14091.0,13745.0,13509.0,13248.0,4478.0,4274.0,4334.0,4143.0,2003
45,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Production occupations,10081.0,9700.0,6863.0,6696.0,6682.0,6566.0,3218.0,3004.0,3144.0,2938.0,2003


# Create BLS Heat Map Dataframe

In [6]:
# Drop df_bls_all_filtered rows that contains 'Total'
df_bls_all_filtered = df_bls_all_filtered[df_bls_all_filtered['occupation'] != 'Total']

bls_data_female_df = df_bls_all_filtered[['year','occupation','female_20_years_and_over']].reset_index()
bls_data_female_df['Sex'] = 'F'
bls_data_female_df.rename(columns={'female_20_years_and_over': 'Count'}, inplace=True)
bls_data_female_df.drop(0, inplace=True)

bls_data_male_df = df_bls_all_filtered[['year','occupation','male_20_years_and_over',]].reset_index()
bls_data_male_df['Sex'] = 'M'
bls_data_male_df.rename(columns={'male_20_years_and_over': 'Count'}, inplace=True)
bls_data_male_df.drop(0, inplace=True)

bls_data_df = pd.concat([bls_data_female_df, bls_data_male_df],    # Combine vertically
                          ignore_index = True,
                          sort = False)

del bls_data_df['index']

# Drop two records from 2010 for which the occupation is "..." and the count is "Unknown"
bls_data_df = bls_data_df[bls_data_df['Count'] != 'Unknown']


bls_data_df = bls_data_df.groupby(['year','occupation','Sex'])['Count'].sum().reset_index()
bls_data_df.rename(columns={'Count': 'BLS_OCCUP_EMPLOYMENT'}, inplace=True)

# Generate total number of students for each calendar year
bls_year_totals = bls_data_df.groupby(['year'])['BLS_OCCUP_EMPLOYMENT'].sum().reset_index()
bls_year_totals.rename(columns={'BLS_OCCUP_EMPLOYMENT': 'BLS_OCCUP_TOTAL_YEAR'}, inplace=True)

# Add percentage of total values for each occupation by year
bls_data_df = pd.merge(bls_data_df, bls_year_totals, how='left', left_on='year',right_on='year')
bls_data_df['BLS_PERCENTAGE']=bls_data_df['BLS_OCCUP_EMPLOYMENT']/bls_data_df['BLS_OCCUP_TOTAL_YEAR']
#bls_data_df['PERCENTAGE']= student_data_df['STUDENTS_IN_MAJOR']/student_data_df['TOTAL_STUDENTS']
#bls_data_df.sort_values(by=['TERM_YEAR','OCCUPATION'],ascending=[False,True])

bls_data_df['year'] = bls_data_df['year'].astype(int)


bls_data_df

,year,occupation,Sex,BLS_OCCUP_EMPLOYMENT,BLS_OCCUP_TOTAL_YEAR,BLS_PERCENTAGE
0,2003,Architecture and engineering occupations,F,382.0,311210.0,0.001227
1,2003,Architecture and engineering occupations,M,2334.0,311210.0,0.0075
2,2003,"Arts, design, entertainment, sports, and media...",F,1223.0,311210.0,0.00393
3,2003,"Arts, design, entertainment, sports, and media...",M,1357.0,311210.0,0.00436
4,2003,Building and grounds cleaning and maintenance ...,F,1956.0,311210.0,0.006285
...,...,...,...,...,...,...
747,2015,Sales and related occupations,M,7565.0,288198.0,0.026249
748,2015,Service occupations,F,13637.0,288198.0,0.047318
749,2015,Service occupations,M,10295.0,288198.0,0.035722
750,2015,Transportation and material moving occupations,F,1341.0,288198.0,0.004653


# Merging the Student and BLS Dataframes

In [7]:
combined_heatmap_df = pd.merge(bls_data_df,student_data_df,how='left', left_on=['year','occupation','Sex'],
                               right_on=['ESTIMATED_COMPLETION_YEAR','OCCUPATION','SEX'])
combined_heatmap_df['year'] = combined_heatmap_df['year'].astype(int)
combined_heatmap_df = combined_heatmap_df[(combined_heatmap_df['year'] <= 2014) & (combined_heatmap_df['year'] >= 2007)]
combined_heatmap_df.drop(['TERM_YEAR','OCCUPATION','SEX'], axis=1, inplace=True)
combined_heatmap_df['BLS% - Degrees%'] = combined_heatmap_df['BLS_PERCENTAGE']-combined_heatmap_df['PERCENTAGE']
combined_heatmap_df = combined_heatmap_df.rename(columns={'year': 'Year', 'occupation': 'Occupation', 'BLS_PERCENTAGE': 'BLS%', 'PERCENTAGE': 'Degrees%'})
combined_heatmap_df['Sex']=['Female' if letter == 'F' else 'Male' for letter in combined_heatmap_df['Sex']]

combined_heatmap_df['Gender_Delta'] = combined_heatmap_df.groupby(['Year','Occupation'])['Degrees%'].diff()
combined_heatmap_df=combined_heatmap_df.reset_index(drop=True)
combined_heatmap_df['Gender_Delta'] = [-combined_heatmap_df['Gender_Delta'][i+1] if i%2 == 0 else
                                       combined_heatmap_df['Gender_Delta'][i] for i in range(len(combined_heatmap_df['Gender_Delta']))]

combined_heatmap_df


,Year,Occupation,Sex,BLS_OCCUP_EMPLOYMENT,BLS_OCCUP_TOTAL_YEAR,BLS%,STUDENTS_IN_RELATED_MAJOR,ESTIMATED_COMPLETION_YEAR,TOTAL_STUDENTS_TERM_YEAR,Degrees%,BLS% - Degrees%,Gender_Delta
0,2007,Architecture and engineering occupations,Female,420.0,331696.0,0.001266,274.0,2007.0,5039.0,0.054376,-0.05311,-0.068268
1,2007,Architecture and engineering occupations,Male,2501.0,331696.0,0.00754,618.0,2007.0,5039.0,0.122643,-0.115103,0.068268
2,2007,"Arts, design, entertainment, sports, and media...",Female,1277.0,331696.0,0.00385,204.0,2007.0,5039.0,0.040484,-0.036634,0.006152
3,2007,"Arts, design, entertainment, sports, and media...",Male,1435.0,331696.0,0.004326,173.0,2007.0,5039.0,0.034332,-0.030006,-0.006152
4,2007,Building and grounds cleaning and maintenance ...,Female,2123.0,331696.0,0.0064,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
459,2014,Sales and related occupations,Male,7580.0,339242.0,0.022344,NaN,NaN,NaN,NaN,NaN,NaN
460,2014,Service occupations,Female,13613.0,339242.0,0.040128,NaN,NaN,NaN,NaN,NaN,NaN
461,2014,Service occupations,Male,10360.0,339242.0,0.030539,NaN,NaN,NaN,NaN,NaN,NaN
462,2014,Transportation and material moving occupations,Female,1354.0,339242.0,0.003991,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# import sys
# sys.path.append('/assets/data_manipulation.py')

from data_manipulation import get_df_list_final, get_short_names


# a list of three data frames at different aggregation levels
# 0 = l1, 1 = l2, 2 = l3    
df = get_df_list_final()[0]

df['l1_short_name'] = df.apply(lambda x: get_short_names('l1', 'default')[x['l1']], axis=1)

df[df['l1'] == 'Architecture and engineering occupations']




,year,l1,number_of_students_all_sum,number_of_students_men_sum,number_of_students_women_sum,number_of_students_unknown_sum,number_of_workers_all_sum,median_weekly_earnings_all_mean,number_of_workers_men_sum,median_weekly_earnings_men_mean,number_of_workers_women_sum,median_weekly_earnings_women_mean,l1_short_name
0,2011,Architecture and engineering occupations,1133.0,844.0,289.0,0.0,2493,753.285714,2178,739.904762,315,0.000000,Arch & Eng
22,2012,Architecture and engineering occupations,1196.0,908.0,288.0,0.0,2589,764.095238,2269,782.761905,318,0.000000,Arch & Eng
44,2013,Architecture and engineering occupations,1166.0,860.0,306.0,0.0,2539,739.095238,2207,678.333333,332,44.476190,Arch & Eng
66,2014,Architecture and engineering occupations,1204.0,918.0,286.0,0.0,2530,775.380952,2151,799.809524,377,98.095238,Arch & Eng
88,2015,Architecture and engineering occupations,872.0,635.0,237.0,0.0,2655,819.476190,2273,829.333333,382,108.333333,Arch & Eng


# Creating Dataframe for Heat Maps from Dataframe df

In [9]:
# Build dataframe for heatmaps off of base df
heatmap_final_women_df=df.copy()
heatmap_final_men_df=df.copy()

# Create restructured dataframe with 'Gender' as a feature, first for women
heatmap_final_women_df['Sex']=['Women']*len(heatmap_final_women_df)
heatmap_final_women_df['Students in Related Major']=df['number_of_students_women_sum']
heatmap_final_women_df['Workers in Occupation']=df['number_of_workers_women_sum']

heatmap_final_men_df['Sex']=['Men']*len(heatmap_final_men_df)
heatmap_final_men_df['Students in Related Major']=df['number_of_students_men_sum']
heatmap_final_men_df['Workers in Occupation']=df['number_of_workers_men_sum']

# Combine the separate women and men dataframes vertically and sort
heatmap_final_df = pd.concat([heatmap_final_women_df, heatmap_final_men_df],    
                          ignore_index = True,
                          sort = False)
heatmap_final_df = heatmap_final_df.sort_values(by=['l1','year','Sex'],ascending=[True, True, False])

# Calculate number_of_workers_total and numberof_students_total for each year and add these two features to heatmap_df
totals_df = df.groupby(['year'])[['number_of_students_all_sum','number_of_workers_all_sum']].sum().reset_index()
totals_df = totals_df.rename(columns={'year':'Year','number_of_students_all_sum': 'Number_of_Students_Total_Year', 'number_of_workers_all_sum': 'Number_of_Workers_Total_Year'})
heatmap_final_df = pd.merge(heatmap_final_df, totals_df, how='left', left_on='year',right_on='Year')

# Calculate workers and students proportions for each occupation and sex combination and add these two features to heatmap_df
heatmap_final_df['Students%']=heatmap_final_df['Students in Related Major']/heatmap_final_df['Number_of_Students_Total_Year']
heatmap_final_df['Workers%']=heatmap_final_df['Workers in Occupation']/heatmap_final_df['Number_of_Workers_Total_Year']

# Reorder features and drop unnecessary features
heatmap_final_df = heatmap_final_df[['Year','l1_short_name','Sex','Workers in Occupation','Number_of_Workers_Total_Year','Workers%',
                                     'Students in Related Major', 'Number_of_Students_Total_Year','Students%']]

# Create 'Workers% - Students%' feature 
heatmap_final_df['Workers% - Students%']=heatmap_final_df['Workers%']-heatmap_final_df['Students%']

# Create 'Sex% Delta' features
heatmap_final_df['Sex% Delta (Students)'] = heatmap_final_df.groupby(['Year','l1_short_name'])['Students%'].diff()
heatmap_final_df['Sex% Delta (Students)']=[-heatmap_final_df['Sex% Delta (Students)'][i+1] if i%2 == 0 else
                                       heatmap_final_df['Sex% Delta (Students)'][i] for i in range(len(heatmap_final_df['Sex% Delta (Students)']))]

heatmap_final_df['Sex% Delta (Workers)'] = heatmap_final_df.groupby(['Year','l1_short_name'])['Workers%'].diff()
heatmap_final_df['Sex% Delta (Workers)']=[-heatmap_final_df['Sex% Delta (Workers)'][i+1] if i%2 == 0 else
                                       heatmap_final_df['Sex% Delta (Workers)'][i] for i in range(len(heatmap_final_df['Sex% Delta (Workers)']))]

# Change 'year' feature to integer type
#heatmap_final_df['Year'] = heatmap_final_df['Year'].astype(int)

heatmap_final_df.head()


,Year,l1_short_name,Sex,Workers in Occupation,Number_of_Workers_Total_Year,Workers%,Students in Related Major,Number_of_Students_Total_Year,Students%,Workers% - Students%,Sex% Delta (Students),Sex% Delta (Workers)
0,2011,Arch & Eng,Women,315,100463,0.003135,289.0,6239.0,0.046322,-0.043186,-0.088957,-0.018544
1,2011,Arch & Eng,Men,2178,100463,0.021680,844.0,6239.0,0.135278,-0.113598,0.088957,0.018544
2,2012,Arch & Eng,Women,318,102754,0.003095,288.0,6159.0,0.046761,-0.043666,-0.100666,-0.018987
3,2012,Arch & Eng,Men,2269,102754,0.022082,908.0,6159.0,0.147427,-0.125345,0.100666,0.018987
4,2013,Arch & Eng,Women,332,104255,0.003184,306.0,6456.0,0.047398,-0.044213,-0.085812,-0.017985


# Create Heat Maps Using Altair
### Note: Each separate heat map is created by manually changing the variable/feature used for the color channel

In [10]:
import altair as alt

#domain =[-.1, -.05, 0, .05, .1] # Manipulate color scale for heat maps
#range_ = ['#002352', '#00326e', '#7c7b78', '#b0a472', '#ffe844']

heatmap = alt.Chart(heatmap_final_df).mark_bar(size=20).encode(
        x=alt.X('Year:O'),
        y=alt.Y('l1_short_name:N', axis=alt.Axis(tickCount=5, title='')),
        color=alt.Color('Sex% Delta (Workers):Q',scale=alt.Scale(scheme='cividis'))
        ).properties(width=300, height=600
        ).facet(column=alt.Column('Sex:N',title="")).configure_view(strokeWidth=0
        )


heatmap

alt.FacetChart(...)

# Create New Dataframe with Worker/Student as a Classification Feature

In [23]:
# Create revised dataframe for heat maps based on dataframe df

rev_heatmap_workers_df=df.copy()
rev_heatmap_students_df=df.copy()

# Create restructured dataframe with 'Worker vs Student' classification as a feature, first for women
rev_heatmap_workers_df['Employment Classification']=['Workers']*len(rev_heatmap_workers_df)
rev_heatmap_workers_df['% Women in Sub-Population']=rev_heatmap_workers_df['number_of_workers_women_sum']/rev_heatmap_workers_df['number_of_workers_all_sum']
#heatmap_final_women_df['Workers in Occupation']=df['number_of_workers_women_sum']

rev_heatmap_students_df['Employment Classification']=['Students']*len(rev_heatmap_students_df)
rev_heatmap_students_df['% Women in Sub-Population']=rev_heatmap_students_df['number_of_students_women_sum']/rev_heatmap_workers_df['number_of_students_all_sum']

#heatmap_final_men_df['Workers in Occupation']=df['number_of_workers_men_sum']

# Combine the separate workers and students dataframes vertically and sort
rev_heatmap_df = pd.concat([rev_heatmap_workers_df, rev_heatmap_students_df],    
                          ignore_index = True,
                          sort = False)

rev_heatmap_df['Women Workers %'] = rev_heatmap_df['number_of_workers_women_sum']/rev_heatmap_df['number_of_workers_all_sum']
rev_heatmap_df['Women Students %'] = rev_heatmap_df['number_of_students_women_sum']/rev_heatmap_df['number_of_students_all_sum']
rev_heatmap_df = rev_heatmap_df.sort_values(by=['l1','year','Employment Classification'],ascending=[True, True, False])
rev_heatmap_df = rev_heatmap_df[rev_heatmap_df['number_of_students_all_sum']!=0]



# Create Revised Heat Map

In [24]:


heatmap = alt.Chart(rev_heatmap_df).mark_bar(size=20).encode(
        x=alt.X('year:O'),
        y=alt.Y('l1_short_name:N', axis=alt.Axis(tickCount=5, title='')),
        color=alt.Color('% Women in Sub-Population:Q',scale=alt.Scale(scheme='cividis'))
        ).properties(width=300, height=600
        ).facet(column=alt.Column('Employment Classification:N',title="")).configure_view(strokeWidth=0
        )


heatmap

alt.FacetChart(...)

In [ ]:
# import bokeh 
# bokeh.sampledata.download()

import pandas as pd
from squarify import normalize_sizes, squarify

from bokeh.plotting import figure, show

from bokeh.sampledata.sample_superstore import data
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from jupyter_bokeh.widgets import BokehModel # widgets third party extension will be required

data = data[["City", "Region", "Sales"]]

regions = ("West", "Central", "South", "East")

sales_by_city = data.groupby(["Region", "City"]).sum("Sales")
sales_by_city = sales_by_city.sort_values(by="Sales").reset_index()

sales_by_region = sales_by_city.groupby("Region").sum("Sales").sort_values(by="Sales")

def treemap(df, col, x, y, dx, dy, *, N=100):
    sub_df = df.nlargest(N, col)
    normed = normalize_sizes(sub_df[col], dx, dy)
    blocks = squarify(normed, x, y, dx, dy)
    blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
    return sub_df.join(blocks_df, how='left').reset_index()

x, y, w, h = 0, 0, 800, 450

blocks_by_region = treemap(sales_by_region, "Sales", x, y, w, h)

dfs = []
for index, (Region, Sales, x, y, dx, dy) in blocks_by_region.iterrows():
    df = sales_by_city[sales_by_city.Region==Region]
    dfs.append(treemap(df, "Sales", x, y, dx, dy, N=10))
blocks = pd.concat(dfs)

p = figure(width=w, height=h, tooltips="@City", toolbar_location=None,
           x_axis_location=None, y_axis_location=None)
p.x_range.range_padding = p.y_range.range_padding = 0
p.grid.grid_line_color = None

p.block('x', 'y', 'dx', 'dy', source=blocks, line_width=1, line_color="white",
        fill_alpha=0.8, fill_color=factor_cmap("Region", "MediumContrast4", regions))

p.text('x', 'y', x_offset=2, text="Region", source=blocks_by_region,
       text_font_size="18pt", text_color="white")

blocks["ytop"] = blocks.y + blocks.dy
p.text('x', 'ytop', x_offset=2, y_offset=2, text="City", source=blocks,
       text_font_size="6pt", text_baseline="top",
       text_color=factor_cmap("Region", ("black", "white", "black", "white"), regions))


output_notebook()
handle = display(BokehModel(p))

blocks


In [ ]:
# blocks_by_L2 = treemap(df_bls_all_filtered_L2, "total_16_years_and_over_py", x, y, w, h)
# blocks_by_L2


df_bls_all_filtered = df_bls_all[df_bls_all['occupation'] != 'Unknown']
df_bls_all_filtered_L0 = df_bls_all[(df_bls_all['level'] == 'L0') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]

df_bls_all_filtered_L0_2015 = df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == '2015'][['level', 'occupation', 'total_16_years_and_over_py']]

l0_grouping = df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == '2015'][['l1', 'occupation', 'total_16_years_and_over_py']].groupby(['occupation'],  sort=False).sum()
l1_grouping = df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == '2015'][['l1', 'total_16_years_and_over_py']].groupby(['l1'],  sort=True).sum()

l1_grouping['total_16_years_and_over_py'] =  pd.to_numeric(l1_grouping['total_16_years_and_over_py'], errors='coerce')
l0_grouping['total_16_years_and_over_py'] =  pd.to_numeric(l0_grouping['total_16_years_and_over_py'], errors='coerce')


def treemap(df, col, x, y, dx, dy, *, N=100):
    sub_df = df.nlargest(N, col)
    normed = normalize_sizes(sub_df[col], dx, dy)
    blocks = squarify(normed, x, y, dx, dy)
    blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
    return sub_df.join(blocks_df, how='left').reset_index()


l1s = tuple(df_bls_all_filtered['l1'].unique().tolist())


l1s = ('Sales and office occupations', 'Professional and related occupations', 'Natural resources, construction, and maintenance occupations', 'Management, business, and financial operations occupations', 'Service occupations')

x, y, w, h = 0, 0, 1600, 900
blocks_by_L1 = treemap(l1_grouping, "total_16_years_and_over_py", x, y, w, h)


blocks_by_L1

dfs = []
for index, (l1, head_count, x, y, dx, dy) in blocks_by_L1.iterrows():
    df = l0_grouping[l0_grouping.l1==l1]    
    dfs.append(treemap(df, "total_16_years_and_over_py", x, y, dx, dy, N=100))

blocks = pd.concat(dfs)


p = figure(width=w, height=h, tooltips="@occupation", toolbar_location=None,
           x_axis_location=None, y_axis_location=None)
p.x_range.range_padding = p.y_range.range_padding = 0
p.grid.grid_line_color = None

p.block('x', 'y', 'dx', 'dy', source=blocks, line_width=1, line_color="white",
        fill_alpha=0.8, fill_color=factor_cmap("l1", "MediumContrast5", l1s))

p.text('x', 'y', x_offset=2, y_offset=-50, text="l1", source=blocks_by_L1,
       text_font_size="18pt", text_color="white")


blocks["ytop"] = blocks.y + blocks.dy

p.text('x', 'ytop', x_offset=2, y_offset=2, text="occupation", source=blocks,
       text_font_size="10pt", text_baseline="top",
       text_color='white')


output_notebook()
handle = display(BokehModel(p))
blocks
